In [74]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import pandas as pd
import numpy as np
import nltk
import gensim
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [75]:
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/doc-chatbot/Emotion Phrases.csv',header=None)
df.head()

,0,1
0,joy,[ On days when I feel close to my partner and ...
1,fear,Every time I imagine that someone I love or I ...
2,anger,When I had been obviously unjustly treated and...
3,sadness,When I think about the short time that we live...
4,disgust,At a gathering I found myself involuntarily si...


In [76]:
df.columns=['emotion','phrase']
df.head()

,emotion,phrase
0,joy,[ On days when I feel close to my partner and ...
1,fear,Every time I imagine that someone I love or I ...
2,anger,When I had been obviously unjustly treated and...
3,sadness,When I think about the short time that we live...
4,disgust,At a gathering I found myself involuntarily si...


In [77]:
nltk.download('stopwords')
nltk.download('wordnet')
stop=stopwords.words('english')
lem=WordNetLemmatizer()
#perform preprocessing
def clean(phrase):
  tokens=gensim.utils.simple_preprocess(phrase)

  clean_tokens=[token for token in tokens if token not in stop]
  lemmed_tokens=[lem.lemmatize(word) for word in clean_tokens]
  sentence=' '.join(lemmed_tokens)

  
  return sentence

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [78]:
df['phrase']=df['phrase'].apply(lambda x:clean(x))
df.head()

,emotion,phrase
0,joy,day feel close partner friend feel peace also ...
1,fear,every time imagine someone love could contact ...
2,anger,obviously unjustly treated possibility elucida...
3,sadness,think short time live relate period life think...
4,disgust,gathering found involuntarily sitting next two...


In [79]:
from keras.preprocessing.text import Tokenizer
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 5000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 60
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['phrase'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 7943 unique tokens.


In [80]:
from keras.preprocessing.sequence import pad_sequences
X = tokenizer.texts_to_sequences(df['phrase'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (7652, 60)


In [81]:
Y = pd.get_dummies(df['emotion']).values

print('Shape of label tensor:', Y.shape)

Shape of label tensor: (7652, 7)


In [82]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 0)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)


(6121, 60) (6121, 7)
(1531, 60) (1531, 7)


In [83]:
size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

7944


In [84]:
import random
from keras.models import *
from keras.layers import *
from keras.callbacks import *

model=Sequential()

#embedding layer
model.add(Embedding(size_of_vocabulary,100,input_length=100,trainable=True)) 

#lstm layer
model.add(LSTM(200,return_sequences=True,dropout=0.2))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(124,activation='relu'))
model.add(Dense(64,activation='relu')) 
model.add(Dense(7,activation='softmax')) 

#Add loss function, metrics, optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=["acc"]) 

#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  

#Print summary of model
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 100)          794400    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 200)          240800    
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 200)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 124)               24924     
_________________________________________________________________
dense_9 (Dense)              (None, 64)                8000      
_________________________________________________________________
dense_10 (Dense)             (None, 7)                 455       
Total params: 1,068,579
Trainable params: 1,068,579
Non-trainable params: 0
____________________________________________

In [85]:
history = model.fit(np.array(X_train),np.array(Y_train),batch_size=128,epochs=30,validation_data=(np.array(X_test),np.array(Y_test)),verbose=1,callbacks=[es,mc])

Epoch 1/30
48/48 [==============================] - ETA: 0s - loss: 1.9427 - acc: 0.1838WARNING:tensorflow:Model was constructed with shape (None, 100) for input Tensor("embedding_3_input:0", shape=(None, 100), dtype=float32), but it was called on an input with incompatible shape (None, 60).

Epoch 00001: val_acc improved from -inf to 0.30960, saving model to best_model.h5
48/48 [==============================] - 19s 391ms/step - loss: 1.9427 - acc: 0.1838 - val_loss: 1.9243 - val_acc: 0.3096
Epoch 2/30
48/48 [==============================] - ETA: 0s - loss: 1.7289 - acc: 0.3611
Epoch 00002: val_acc improved from 0.30960 to 0.39647, saving model to best_model.h5
48/48 [==============================] - 18s 380ms/step - loss: 1.7289 - acc: 0.3611 - val_loss: 1.5481 - val_acc: 0.3965
Epoch 3/30
48/48 [==============================] - ETA: 0s - loss: 1.2082 - acc: 0.5708
Epoch 00003: val_acc improved from 0.39647 to 0.51078, saving model to best_model.h5
48/48 [=========================

In [86]:
#loading best model
from keras.models import load_model
model = load_model('best_model.h5')

#evaluation 
_,val_acc = model.evaluate(X_test,Y_test, batch_size=128)
print(val_acc)

12/12 [==============================] - 1s 105ms/step - loss: 1.4324 - acc: 0.5251
0.5251469612121582


In [87]:
questions=[['Why are you are coming in for an appointment now?.'],
['What are the particular things that are bothering you?.'],
['What would you like help with?.'],
['How far are you from your happiness?.'],
['How would you describe your normal personality? Are you more a cup half-full person or cup half-empty?.'],
['When did you start feeling bad?.'],
['When was the last time you felt happy?.']]

In [88]:
sent_list=[]
emotion_list=[]

def response(user_responce):
    
    sent=user_responce
    #sent_list.append(sent)
    
    seq = tokenizer.texts_to_sequences([sent])
    padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
    pred = model.predict(padded)

    #labels = ['Credit reporting, credit repair services, or other personal consumer reports', 'Debt collection', 'Mortgage', 'Credit card or prepaid card', 'Student loan', 'Bank account or service', 'Checking or savings account', 'Consumer Loan', 'Payday loan, title loan, or personal loan', 'Vehicle loan or lease', 'Money transfer, virtual currency, or money service', 'Money transfers', 'Prepaid card']
    labels=['anger','disgust','fear','guilt','joy','sadness','shame']
    #emotion_list.append(emotion)
    return  labels[np.argmax(pred)]
    

In [89]:
print("DOC:Hi you can call me doc and  I am here to help you with the some stress isuues so let me ask you some simmple questions, ok?")
user_response = input()
if(user_response=='yes doc' or user_response=='hello doc' or 'Hi doc' or 'hi doc' or 'Hi doctor' or 'Ok doc' or 'ok Doc' ):
    
    
    print("DOC: good lets start the session..")
for i in range(0,5):
    print(questions[i])
    user_response = input()
    user_response=user_response.lower()

   
      
    print(response(user_response))
    
    
    if emotion =='joy':
      print('DOC-'+'ohh nice its really good thing')
    else:
      
      doc_res = ['ohh its ok we can deal with that', 'Thats why i am here i can healp you with that', 'you need to not to feel like that its just small issue', 'Its common at your age', 'you need not be sad ok, i am here for you']

      rand_res = random.choice(doc_res)
      print ("Doc - ", rand_res)
    


print("DOC: Bye! Stay cool and follow the prescription.")

DOC:Hi you can call me doc and  I am here to help you with the some stress isuues so let me ask you some simmple questions, ok?
ok doc
DOC: good lets start the session..
['Why are you are coming in for an appointment now?.']
feeling lonely and sadd
sadness
Doc -  you need not be sad ok, i am here for you
['What are the particular things that are bothering you?.']
negative thinking and some anger also
anger
Doc -  you need to not to feel like that its just small issue
['What would you like help with?.']
help me with negative thoughts and i am feeling incomplete
guilt
Doc -  ohh its ok we can deal with that
['How far are you from your happiness?.']
to be frank soo far doc its been months i felt really happy
joy
Doc -  you need to not to feel like that its just small issue
['How would you describe your normal personality? Are you more a cup half-full person or cup half-empty?.']
normally i am cool and funny guy because of some health issues now i am a cup half empty
fear
Doc -  ohh its ok

In [92]:
import requests
import json

class telegram_bot():
    def __init__(self):
        self.token = "1305125892:AAFknVI_m4mHgYnE-BTlAYn3Naa0ZKTV3zk"    #write your token here!  
    
        self.url = f"https://api.telegram.org/bot{self.token}"

    def get_updates(self,offset=None):
        url = self.url+"/getUpdates?timeout=100"
        if offset:
            url = url+f"&offset={offset+1}"
        url_info = requests.get(url)
        return json.loads(url_info.content)

    def send_message(self,msg,chat_id):
        url = self.url + f"/sendMessage?chat_id={chat_id}&text={msg}"
        if msg is not None:
            requests.get(url)

    def grab_token(self):
        return tokens

In [98]:
tbot = telegram_bot()

update_id = None

def make_reply(msg):
    if msg.lower() =='hi doc':
        print("DOC:Hi I am here to help you with the some stress isuues so let me ask you some simmple questions, ok?")
        user_response = msg.lower()
        if(user_response=='yes doc' or user_response=='hello doc' or 'Hi doc' or 'hi doc' or 'Hi doctor' or 'Ok doc' or 'ok Doc' ):
            
            
            reply=("DOC: good lets start the session..")
        for i in range(0,5):
            reply=(questions[i])
            

            #user_response = input()
            #user_response=user_response.lower()

          
              
            emotion=response(user_response)
            
            
            if emotion =='joy':
              reply='DOC-'+'ohh nice its really good thing'
            else:
              
              doc_res = ['ohh its ok we can deal with that', 'Thats why i am here i can healp you with that', 'you need to not to feel like that its just small issue', 'Its common at your age', 'you need not be sad ok, i am here for you']

              rand_res = random.choice(doc_res)
              reply= rand_res
            


        reply="DOC: Bye! Stay cool and follow the prescription."
        return reply,doc1,emotion
while True:
    print("...")
    updates = tbot.get_updates(offset=update_id)
    updates = updates['result']
    print(updates)
    if updates:
        for item in updates:
            update_id = item["update_id"]
            print(update_id)
            try:
                message = item["message"]["text"]
                print(message)
            except:
                message = None
            from_ = item["message"]["from"]["id"]
            print(from_)

            reply = make_reply(message)
            tbot.send_message(reply,from_)
            

...
[{'update_id': 11150601, 'message': {'message_id': 47, 'from': {'id': 1352009815, 'is_bot': False, 'first_name': 'Anish', 'last_name': 'T', 'username': 'dicaprio1234', 'language_code': 'en'}, 'chat': {'id': 1352009815, 'first_name': 'Anish', 'last_name': 'T', 'username': 'dicaprio1234', 'type': 'private'}, 'date': 1600888052, 'text': 'Ok doc'}}]
11150601
Ok doc
1352009815
...
[]
...
[]
...
[]
...
[]
...
[]
...
[]
...
[]
...
[{'update_id': 11150602, 'message': {'message_id': 49, 'from': {'id': 1352009815, 'is_bot': False, 'first_name': 'Anish', 'last_name': 'T', 'username': 'dicaprio1234', 'language_code': 'en'}, 'chat': {'id': 1352009815, 'first_name': 'Anish', 'last_name': 'T', 'username': 'dicaprio1234', 'type': 'private'}, 'date': 1600888413, 'text': 'Hi doc'}}]
11150602
Hi doc
1352009815
...
[{'update_id': 11150603, 'message': {'message_id': 51, 'from': {'id': 1352009815, 'is_bot': False, 'first_name': 'Anish', 'last_name': 'T', 'username': 'dicaprio1234', 'language_code': 'en'

KeyboardInterrupt: ignored